# ETL Pipeline Preparation
Follow the instructions below to help you create your ETL pipeline.
### 1. Import libraries and load datasets.
- Import Python libraries
- Load `messages.csv` into a dataframe and inspect the first few lines.
- Load `categories.csv` into a dataframe and inspect the first few lines.

In [1]:
# import libraries
import pandas as pd
import sqlite3
import numpy as np
from sqlalchemy import create_engine
import os

In [2]:
# installing execution time measurement tools 
!pip install ipython-autotime
%load_ext autotime

time: 281 µs (started: 2021-01-05 02:20:30 +01:00)


In [3]:
# load messages dataset
messages = pd.read_csv('data/messages.csv')
print(f'Shape of `messages` df : {messages.shape}')
messages.head()

Shape of `messages` df : (26248, 4)
time: 52.1 ms (started: 2021-01-05 02:20:30 +01:00)


,id,message,original,genre
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct


In [4]:
# load categories dataset
categories =  pd.read_csv('data/categories.csv')
print(f'Shape of `categories` df : {categories.shape}')
categories.head()

Shape of `categories` df : (26248, 2)
time: 52.6 ms (started: 2021-01-05 02:20:30 +01:00)


,id,categories
0,2,related-1;request-0;offer-0;aid_related-0;medi...
1,7,related-1;request-0;offer-0;aid_related-1;medi...
2,8,related-1;request-0;offer-0;aid_related-0;medi...
3,9,related-1;request-1;offer-0;aid_related-1;medi...
4,12,related-1;request-0;offer-0;aid_related-0;medi...


### 2. Merge datasets.
- Merge the messages and categories datasets using the common id
- Assign this combined dataset to `df`, which will be cleaned in the following steps

In [5]:
# merge datasets
df = messages.merge(categories, how='left', on='id')
print(f'Shape after merging : {df.shape}')
df.head()

Shape after merging : (26386, 5)
time: 11.9 ms (started: 2021-01-05 02:20:30 +01:00)


,id,message,original,genre,categories
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,related-1;request-0;offer-0;aid_related-0;medi...
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,related-1;request-0;offer-0;aid_related-1;medi...
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,related-1;request-0;offer-0;aid_related-0;medi...
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,related-1;request-1;offer-0;aid_related-1;medi...
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,related-1;request-0;offer-0;aid_related-0;medi...


### 3. Split `categories` into separate category columns.
- Split the values in the `categories` column on the `;` character so that each value becomes a separate column. You'll find [this method](https://pandas.pydata.org/pandas-docs/version/0.23/generated/pandas.Series.str.split.html) very helpful! Make sure to set `expand=True`.
- Use the first row of categories dataframe to create column names for the categories data.
- Rename columns of `categories` with new column names.

In [6]:
# create a dataframe of the 36 individual category columns
categories = df['categories'].str.split(';',expand=True)
categories.head()

,0,1,2,3,4,5,6,7,8,9,...,26,27,28,29,30,31,32,33,34,35
0,related-1,request-0,offer-0,aid_related-0,medical_help-0,medical_products-0,search_and_rescue-0,security-0,military-0,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-0,floods-0,storm-0,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0
1,related-1,request-0,offer-0,aid_related-1,medical_help-0,medical_products-0,search_and_rescue-0,security-0,military-0,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-1,floods-0,storm-1,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0
2,related-1,request-0,offer-0,aid_related-0,medical_help-0,medical_products-0,search_and_rescue-0,security-0,military-0,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-0,floods-0,storm-0,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0
3,related-1,request-1,offer-0,aid_related-1,medical_help-0,medical_products-1,search_and_rescue-0,security-0,military-0,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-0,floods-0,storm-0,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0
4,related-1,request-0,offer-0,aid_related-0,medical_help-0,medical_products-0,search_and_rescue-0,security-0,military-0,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-0,floods-0,storm-0,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0


time: 182 ms (started: 2021-01-05 02:20:30 +01:00)


In [7]:
# select the first row of the categories dataframe
row = categories.iloc[-1]

# use this row to extract a list of new column names for categories.
# one way is to apply a lambda function that takes everything 
# up to the second to last character of each string with slicing
prefix='category:'
category_colnames = row.apply(lambda col_name: prefix + col_name[:-2]).values
print(category_colnames)

['category:related' 'category:request' 'category:offer'
 'category:aid_related' 'category:medical_help'
 'category:medical_products' 'category:search_and_rescue'
 'category:security' 'category:military' 'category:child_alone'
 'category:water' 'category:food' 'category:shelter' 'category:clothing'
 'category:money' 'category:missing_people' 'category:refugees'
 'category:death' 'category:other_aid' 'category:infrastructure_related'
 'category:transport' 'category:buildings' 'category:electricity'
 'category:tools' 'category:hospitals' 'category:shops'
 'category:aid_centers' 'category:other_infrastructure'
 'category:weather_related' 'category:floods' 'category:storm'
 'category:fire' 'category:earthquake' 'category:cold'
 'category:other_weather' 'category:direct_report']
time: 782 µs (started: 2021-01-05 02:20:30 +01:00)


In [8]:
# rename the columns of `categories`
categories.columns = category_colnames
categories.head()

,category:related,category:request,category:offer,category:aid_related,category:medical_help,category:medical_products,category:search_and_rescue,category:security,category:military,category:child_alone,...,category:aid_centers,category:other_infrastructure,category:weather_related,category:floods,category:storm,category:fire,category:earthquake,category:cold,category:other_weather,category:direct_report
0,related-1,request-0,offer-0,aid_related-0,medical_help-0,medical_products-0,search_and_rescue-0,security-0,military-0,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-0,floods-0,storm-0,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0
1,related-1,request-0,offer-0,aid_related-1,medical_help-0,medical_products-0,search_and_rescue-0,security-0,military-0,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-1,floods-0,storm-1,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0
2,related-1,request-0,offer-0,aid_related-0,medical_help-0,medical_products-0,search_and_rescue-0,security-0,military-0,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-0,floods-0,storm-0,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0
3,related-1,request-1,offer-0,aid_related-1,medical_help-0,medical_products-1,search_and_rescue-0,security-0,military-0,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-0,floods-0,storm-0,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0
4,related-1,request-0,offer-0,aid_related-0,medical_help-0,medical_products-0,search_and_rescue-0,security-0,military-0,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-0,floods-0,storm-0,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0


time: 25.9 ms (started: 2021-01-05 02:20:30 +01:00)


### 4. Convert category values to just numbers 0 or 1.
- Iterate through the category columns in df to keep only the last character of each string (the 1 or 0). For example, `related-0` becomes `0`, `related-1` becomes `1`. Convert the string to a numeric value.
- You can perform [normal string actions on Pandas Series](https://pandas.pydata.org/pandas-docs/stable/text.html#indexing-with-str), like indexing, by including `.str` after the Series. You may need to first convert the Series to be of type string, which you can do with `astype(str)`.

In [9]:
for column in categories:
    # set each value to be the last character of the string
    categories[column] = categories[column].apply(lambda col_name: col_name[-1:]).values
    
    # convert column from string to numeric
    categories[column] = pd.to_numeric(categories[column])
    
categories.head()


,category:related,category:request,category:offer,category:aid_related,category:medical_help,category:medical_products,category:search_and_rescue,category:security,category:military,category:child_alone,...,category:aid_centers,category:other_infrastructure,category:weather_related,category:floods,category:storm,category:fire,category:earthquake,category:cold,category:other_weather,category:direct_report
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,1,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,1,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


time: 918 ms (started: 2021-01-05 02:20:30 +01:00)


In [10]:
# make sure that each category only 1 and 0
for column in categories:
    if len(np.unique(categories[column])) != 2:
        print(column, np.unique(categories[column]))

category:related [0 1 2]
category:child_alone [0]
time: 12.6 ms (started: 2021-01-05 02:20:31 +01:00)


=> the column `category:related` has only not binary value. Therefore we could remove the rows with the value `2`.

In [11]:
categories = categories[categories['category:related'] != 2]

time: 6.58 ms (started: 2021-01-05 02:20:31 +01:00)


=> the column `category:child_alone` has only 0 value. Therefore we could drop this column.

In [12]:
categories.drop('category:child_alone', axis = 1, inplace = True)

time: 1.49 ms (started: 2021-01-05 02:20:31 +01:00)


### 5. Replace `categories` column in `df` with new category columns.
- Drop the categories column from the df dataframe since it is no longer needed.
- Concatenate df and categories data frames.

In [13]:
# drop the original categories column from `df`
df.drop(columns=['categories'], inplace=True)
df.head()

,id,message,original,genre
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct


time: 12.2 ms (started: 2021-01-05 02:20:31 +01:00)


In [14]:
# concatenate the original dataframe with the new `categories` dataframe
df = pd.concat([df, categories], axis=1)
df.head()

,id,message,original,genre,category:related,category:request,category:offer,category:aid_related,category:medical_help,category:medical_products,...,category:aid_centers,category:other_infrastructure,category:weather_related,category:floods,category:storm,category:fire,category:earthquake,category:cold,category:other_weather,category:direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1.0,1.0,0.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


time: 15.3 ms (started: 2021-01-05 02:20:31 +01:00)


In [15]:
df['genre'].unique()

array(['direct', 'social', 'news'], dtype=object)

time: 1.95 ms (started: 2021-01-05 02:20:31 +01:00)


### 6. Remove duplicates.
- Check how many duplicates are in this dataset.
- Drop the duplicates.
- Confirm duplicates were removed.

In [16]:
# check number of duplicates
df[df.duplicated()].count()

id                                 170
message                            170
original                            76
genre                              170
category:related                   154
category:request                   154
category:offer                     154
category:aid_related               154
category:medical_help              154
category:medical_products          154
category:search_and_rescue         154
category:security                  154
category:military                  154
category:water                     154
category:food                      154
category:shelter                   154
category:clothing                  154
category:money                     154
category:missing_people            154
category:refugees                  154
category:death                     154
category:other_aid                 154
category:infrastructure_related    154
category:transport                 154
category:buildings                 154
category:electricity     

time: 48.1 ms (started: 2021-01-05 02:20:31 +01:00)


In [17]:
# drop duplicates
df.drop_duplicates(inplace=True)

time: 26.7 ms (started: 2021-01-05 02:20:31 +01:00)


In [18]:
# check number of duplicates
df[df.duplicated()].count()

id                                 0
message                            0
original                           0
genre                              0
category:related                   0
category:request                   0
category:offer                     0
category:aid_related               0
category:medical_help              0
category:medical_products          0
category:search_and_rescue         0
category:security                  0
category:military                  0
category:water                     0
category:food                      0
category:shelter                   0
category:clothing                  0
category:money                     0
category:missing_people            0
category:refugees                  0
category:death                     0
category:other_aid                 0
category:infrastructure_related    0
category:transport                 0
category:buildings                 0
category:electricity               0
category:tools                     0
c

time: 28.6 ms (started: 2021-01-05 02:20:32 +01:00)


### 7. Save the clean dataset into an sqlite database.
You can do this with pandas [`to_sql` method](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.to_sql.html) combined with the SQLAlchemy library. Remember to import SQLAlchemy's `create_engine` in the first cell of this notebook to use it below.

In [19]:
# create db folder
db_base_folder='db'
if not os.path.exists(db_base_folder):
    os.makedirs(db_base_folder)

# write df to sqllite-db
engine = create_engine(f'sqlite:///{db_base_folder}/disaster_messages_db.db')
df.to_sql('disaster_messages', if_exists='replace', con=engine, index=False)

time: 953 ms (started: 2021-01-05 02:20:32 +01:00)


### 8. Use this notebook to complete `etl_pipeline.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database based on new datasets specified by the user. Alternatively, you can complete `etl_pipeline.py` in the classroom on the `Project Workspace IDE` coming later.

In [20]:
# see `data/process_data.py`

time: 241 µs (started: 2021-01-05 02:20:32 +01:00)
